In [1]:
%update_token

Token successfully refreshed.


In [2]:
from mlflow import MlflowClient
client = MlflowClient()
exp_id = client.get_experiment_by_name("license-detector").experiment_id
artifact_url = client.search_runs(exp_id,order_by=["start_time DESC"])[0].info.artifact_uri + '/triton/triton_engines' # (e.g. mlflow.sy6s171r10/13/d7f6827cd28a4783abfe7866c9e06255/artifacts/triton/triton_engines )
image_tag = "nvcr.io/nvidia/tritonserver:25.02-py3"
isvc_name = "license-plate-detector-engines"
secret_name = "triton-s3-proxy"
service_account = "s3-proxy-kserve-sa"
# local_s3_proxy_uri = "http://local-s3-service.ezdata-system.svc.cluster.local:30000"
with open('/etc/secrets/ezua/.auth_token','r') as file:
    AUTH_TOKEN = file.read().strip()

In [3]:
print(artifact_url)
print(AUTH_TOKEN)

s3://mlflow.sy6s171r10/16/3c729e661678484fa6048ca2dead4dfe/artifacts
eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJneExEamtYbDBTTHNGWGp6TXFPQ3E4bjEzdmJ4RXhmeUM1M1lCOGVCRGdzIn0.eyJleHAiOjE3NDg1MDUzNzYsImlhdCI6MTc0ODUwMzU3NiwiYXV0aF90aW1lIjoxNzQ4Mzk2NDcxLCJqdGkiOiI1MDBjOWNkYS04YmYxLTQ1NzctOTdkNC1mMmNhMDFjNjNjMzMiLCJpc3MiOiJodHRwczovL2tleWNsb2FrLmluZ3Jlc3MucGNhaTAxMDMuc3k2LmhwZWNvbG8ubmV0L3JlYWxtcy9VQSIsInN1YiI6IjZlMDY0YTc0LTAyZmEtNGZjOC1iNTk0LThiYzgwMDE4MDQ4YSIsInR5cCI6IkJlYXJlciIsImF6cCI6InVhIiwibm9uY2UiOiJVMEVtQXNfRXFsV0stOHUyZ0FUeDRaMmJTVjNvTnc2ZVRxeUY2OXVic2hrIiwic2Vzc2lvbl9zdGF0ZSI6ImZjNDAyMTJhLTBjZDItNDZhMC1hZjJkLTFkZDY5ZmJiMWRkOCIsImFjciI6IjEiLCJzY29wZSI6Im9wZW5pZCBlbWFpbCBwcm9maWxlIG9mZmxpbmVfYWNjZXNzIHN5NnMxNzFyMTA6dWEiLCJzaWQiOiJmYzQwMjEyYS0wY2QyLTQ2YTAtYWYyZC0xZGQ2OWZiYjFkZDgiLCJ1aWQiOiIxMDAwMDAwOCIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiZ2lkIjoiMTAwMSIsIm5hbWUiOiJHZXVuIFRhayBSb2giLCJuYW1lc3BhY2UiOiJnZXVuLXRhay1yb2gtaHAtNTdmNjlkNDciLCJncm91cHMiOlsidWEtZW5hYmxlZCIsIm9mZmxpbmVfYWN

In [4]:
with open(isvc_name + '.yaml', 'w') as file:
    text = f"""---
apiVersion: serving.kserve.io/v1alpha1
kind: ServingRuntime
metadata:
  generation: 1
  name: kserve-tritonserver-ngc
spec:
  annotations:
    prometheus.kserve.io/path: /metrics
    prometheus.kserve.io/port: "8002"
  containers:
  - args:
    - tritonserver
    - --model-store=/mnt/models
    - --grpc-port=9000
    - --http-port=8080
    - --allow-grpc=true
    - --allow-http=true
    image: {image_tag}
    name: kserve-container
    resources:
      limits:
        cpu: "1"
        memory: 2Gi
      requests:
        cpu: "1"
        memory: 2Gi
  protocolVersions:
  - v2
  - grpc-v2
  supportedModelFormats:
  - autoSelect: true
    name: tensorrt
    priority: 1
    version: "8"
  - autoSelect: true
    name: tensorflow
    priority: 1
    version: "1"
  - autoSelect: true
    name: tensorflow
    priority: 1
    version: "2"
  - autoSelect: true
    name: onnx
    priority: 1
    version: "1"
  - name: pytorch
    version: "1"
  - autoSelect: true
    name: triton
    priority: 1
    version: "2"
---
apiVersion: v1
kind: Secret
metadata:
  name: "{secret_name}"
  annotations:
    serving.kserve.io/s3-cabundle: ""
    serving.kserve.io/s3-endpoint: "local-s3-service.ezdata-system.svc.cluster.local:30000/"
    serving.kserve.io/s3-useanoncredential: "false"
    serving.kserve.io/s3-usehttps: "0"
    serving.kserve.io/s3-verifyssl: "0"
stringData:
  AWS_ACCESS_KEY_ID: "{AUTH_TOKEN}"
  AWS_SECRET_ACCESS_KEY: "s3"
type: Opaque
---
apiVersion: v1
kind: ServiceAccount
metadata:
  name: "{service_account}"
secrets:
  - name: "{secret_name}"
---
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: {isvc_name}
spec:
  predictor:
    serviceAccountName: "{service_account}"
    model:
      modelFormat:
        name: triton
      storageUri: "{artifact_url}"
      runtime: kserve-tritonserver-ngc
      resources:
        limits:
          nvidia.com/gpu: 1
        requests:
          nvidia.com/gpu: 1
"""
    file.write(text)

In [5]:
!ls -rlt license-plate-detector-engines.yaml

-rw-r--r--. 1 geun-tak.roh-hpe.com group-1001 3609 May 29 07:35 license-plate-detector-engines.yaml


In [6]:
!kubectl apply -f license-plate-detector-engines.yaml

servingruntime.serving.kserve.io/kserve-tritonserver-ngc unchanged
secret/triton-s3-proxy configured
serviceaccount/s3-proxy-kserve-sa configured
inferenceservice.serving.kserve.io/license-plate-detector-engines created


In [8]:
!kubectl get pods

NAME                                                              READY   STATUS    RESTARTS     AGE
copy-0                                                            2/2     Running   0            75d
fs-57c764ddbd-skbzq                                               2/2     Running   0            77d
license-plate-detector-engines-predictor-00001-deployment-sj2zp   1/3     Error     1 (5s ago)   19s
ml-pipeline-ui-artifact-74d66ccfdf-vqwf7                          2/2     Running   0            77d
ml-pipeline-visualizationserver-7794858c98-c5pr5                  2/2     Running   0            77d
triton-kserve-test-0                                              2/2     Running   0            71d
trt-test-0                                                        2/2     Running   0            29h


In [1]:
!kubectl get svc

NAME                                                     TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                                              AGE
copy                                                     ClusterIP   10.99.135.123    <none>        80/TCP                                               75d
fs                                                       ClusterIP   10.108.51.199    <none>        80/TCP                                               77d
license-plate-detector-engines-predictor-00001           ClusterIP   10.96.184.66     <none>        80/TCP,443/TCP                                       3m51s
license-plate-detector-engines-predictor-00001-private   ClusterIP   10.99.236.176    <none>        80/TCP,443/TCP,9090/TCP,9091/TCP,8022/TCP,8012/TCP   3m51s
ml-pipeline-ui-artifact                                  ClusterIP   10.96.21.28      <none>        80/TCP                                               77d
ml-pipeline-visualizationserver                       